<a href="https://colab.research.google.com/github/Aldevand/buzzer-detecction-lstm/blob/main/buzzer_detection_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Daftar Library

In [ ]:
!pip install Sastrawi
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import cosine_similarity

from scipy.spatial.distance import cosine
import networkx as nx
import datetime
import tensorflow as tf
# import torch # Uncomment if you plan to use PyTorch
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# Import Dataset

In [ ]:
import pandas as pd
data = pd.read_excel('/content/drive/MyDrive/Keperluan Skripsi/danantara_gabungan.xlsx')
display(data.head())
display(data.tail())

,conversation_id_str,created_at,favorite_count,full_text,id_str,image_url,in_reply_to_screen_name,lang,location,quote_count,reply_count,retweet_count,tweet_url,user_id_str,username
0,1900102038295633920,Thu Mar 13 08:30:02 +0000 2025,2,Mengacu dari Indonesia Salary Guide 2025 gaji ...,1900102038295633920,NaN,NaN,in,Indonesia,0,0,4,https://x.com/tempodotco/status/19001020382956...,18129942,tempodotco
1,1900122136087585024,Thu Mar 13 09:49:54 +0000 2025,0,Electrizen Danantara Indonesia telah resmi dib...,1900122136087585024,https://pbs.twimg.com/media/Gl6XAlLbYAI-UqI.jpg,NaN,in,NaN,0,0,0,https://x.com/ImeldaTriS54261/status/190012213...,1849369709390807040,ImeldaTriS54261
2,1900086851337413120,Thu Mar 13 07:29:41 +0000 2025,0,Wamenkeu Thomas: Danantara Tak Gadai Saham Pem...,1900086851337413120,NaN,NaN,in,Jakarta Capital Region,0,0,0,https://x.com/cnbcindonesia/status/19000868513...,847372542830444544,cnbcindonesia
3,1898156438125768960,Fri Mar 07 23:38:55 +0000 2025,1,Proyek DME: Mahal &amp; Merugikan! Gasifikasi ...,1898156438125768960,https://pbs.twimg.com/media/GlebOJZbwAIeiQi.jpg,NaN,in,Indonesia,0,1,2,https://x.com/350Indonesia/status/189815643812...,457187389,350Indonesia
4,1899762836152155904,Wed Mar 12 10:02:10 +0000 2025,1,Masyarakat Indonesia percaya bahwa BPI Dananta...,1899762836152155904,https://pbs.twimg.com/media/Gl1QNK1W8AAydDa.jpg,NaN,in,Jakarta,1,1,0,https://x.com/liputan6dotcom/status/1899762836...,47596019,liputan6dotcom


,conversation_id_str,created_at,favorite_count,full_text,id_str,image_url,in_reply_to_screen_name,lang,location,quote_count,reply_count,retweet_count,tweet_url,user_id_str,username
1250,1898963517765820928,Mon Mar 10 05:05:58 +0000 2025,1,Danantara Masuk 10 Besar Badan Investasi Dunia...,1898963517765820928,https://pbs.twimg.com/media/Glp5QVcbcAAMBft.jpg,NaN,in,"Kota Jambi, Jambi",0,1,0,https://x.com/kharism79/status/189896351776582...,1564436564855235072,kharism79
1251,1898005540732912128,Fri Mar 07 13:39:18 +0000 2025,372,Bapak Presiden Jenderal TNI (Purn) Prabowo Sub...,1898005540732912128,https://pbs.twimg.com/media/GlcR8fFbwAAT4vZ.jpg,NaN,in,Indonesia,30,359,39,https://x.com/erickthohir/status/1898005540732...,43088704,erickthohir
1252,1899692230496980992,Wed Mar 12 05:21:36 +0000 2025,1,Mengembangkan Investasi Berbasis Teknologi den...,1899692230496980992,https://pbs.twimg.com/media/Gl0QA5maMAAZc9N.jpg,NaN,in,NaN,0,0,0,https://x.com/pejuang_berkuda/status/189969223...,1645196114453209088,pejuang_berkuda
1253,1900068057382608896,Thu Mar 13 06:15:01 +0000 2025,0,Masyarakat Indonesia percaya bahwa BPI DANANTA...,1900068057382608896,https://pbs.twimg.com/media/Gl5l1ByWIAAVckf.jpg,NaN,in,"Tangerang, Banten",0,0,0,https://x.com/anonymsmee/status/19000680573826...,1533739189124349952,anonymsmee
1254,1900065026184261888,Thu Mar 13 06:02:58 +0000 2025,0,Masyarakat Indonesia percaya bahwa BPI DANANTA...,1900065026184261888,https://pbs.twimg.com/media/Gl5jD2EbYAAkY_v.jpg,NaN,in,"Bandung Wetan, Indonesia",0,0,0,https://x.com/Tivannni/status/1900065026184261840,1686276192352271872,Tivannni


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1255 entries, 0 to 1254
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   conversation_id_str      1255 non-null   int64 
 1   created_at               1255 non-null   object
 2   favorite_count           1255 non-null   int64 
 3   full_text                1255 non-null   object
 4   id_str                   1255 non-null   int64 
 5   image_url                672 non-null    object
 6   in_reply_to_screen_name  112 non-null    object
 7   lang                     1255 non-null   object
 8   location                 810 non-null    object
 9   quote_count              1255 non-null   int64 
 10  reply_count              1255 non-null   int64 
 11  retweet_count            1255 non-null   int64 
 12  tweet_url                1255 non-null   object
 13  user_id_str              1255 non-null   int64 
 14  username                 1255 non-null  

# Seleksi Fitur

In [ ]:
df = data[['full_text','created_at','username','conversation_id_str']].copy() # Menggunakan .copy()

# Pastikan kolom 'created_at' dalam format datetime segera setelah membuat copy
df['created_at'] = pd.to_datetime(df['created_at'])

display(df.head(10))

/tmp/ipython-input-2167281568.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['created_at'] = pd.to_datetime(df['created_at'])


,full_text,created_at,username,conversation_id_str
0,Mengacu dari Indonesia Salary Guide 2025 gaji ...,2025-03-13 08:30:02+00:00,tempodotco,1900102038295633920
1,Electrizen Danantara Indonesia telah resmi dib...,2025-03-13 09:49:54+00:00,ImeldaTriS54261,1900122136087585024
2,Wamenkeu Thomas: Danantara Tak Gadai Saham Pem...,2025-03-13 07:29:41+00:00,cnbcindonesia,1900086851337413120
3,Proyek DME: Mahal &amp; Merugikan! Gasifikasi ...,2025-03-07 23:38:55+00:00,350Indonesia,1898156438125768960
4,Masyarakat Indonesia percaya bahwa BPI Dananta...,2025-03-12 10:02:10+00:00,liputan6dotcom,1899762836152155904
5,Danantara Indonesia percepat pembiayaan transi...,2025-03-13 00:37:56+00:00,gayoesme,1899983227705786880
6,Dari Danantara Indonesia mendunia https://t.co...,2025-03-13 02:27:52+00:00,AlfaCrysta58412,1900010892936569088
7,Indonesia Bisa Makin Bobrok?? #danantara #tega...,2025-03-09 00:48:44+00:00,realAmienRais,1898536395100300032
8,Danantara badan pengelola investasi yang telah...,2025-03-12 02:54:03+00:00,BangsaNara,1899655094234484992
9,INDONESIA SEMAKIN SURAM... PERTAMAX DANANTARA ...,2025-03-03 11:22:05+00:00,secr3thm4nz56,1896521455472660992


Keseragaman Teks (Text Similarity)

In [ ]:
# TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['full_text'])

In [ ]:
# Consine Similarity
cosine_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
# Hasil perhiutngan TF-IDF & Cosine similarity
print(f"Similarity komentar kedua yang sama mirip dengan komen lainnya:\n{cosine_sim_matrix[10]}")

Similarity komentar kedua yang sama mirip dengan komen lainnya:
[0.01118104 0.01503326 0.01142513 ... 0.01072623 0.02128788 0.02128788]


Kemunculan Serentak(Co-occurrence)#

In [ ]:
#-> ubah tipe data crated
df['created_at'] = pd.to_datetime(df['created_at'])

# Buat kolom baru
df['kemunculan_serentak'] = 0

for i, row in df.iterrows():
  waktu_awal = row['created_at']
  waktu_akhir = waktu_awal + pd.Timedelta(minutes=10)
  count = df[(df['created_at'] > waktu_awal) & (df['created_at'] <= waktu_akhir)].shape[0]
  df.loc[i, 'kemunculan_serentak'] = count

print("Dataframe dengan fitur/kolom 'kemunculan serentak':")
print(df[['full_text', 'created_at', 'kemunculan_serentak']])

/tmp/ipython-input-2001516941.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['created_at'] = pd.to_datetime(df['created_at'])
/tmp/ipython-input-2001516941.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['created_at'] = pd.to_datetime(df['created_at'])
/tmp/ipython-input-2001516941.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['kemunculan_serenta

Dataframe dengan fitur/kolom 'kemunculan serentak':
                                              full_text  \
0     Mengacu dari Indonesia Salary Guide 2025 gaji ...   
1     Electrizen Danantara Indonesia telah resmi dib...   
2     Wamenkeu Thomas: Danantara Tak Gadai Saham Pem...   
3     Proyek DME: Mahal &amp; Merugikan! Gasifikasi ...   
4     Masyarakat Indonesia percaya bahwa BPI Dananta...   
...                                                 ...   
1250  Danantara Masuk 10 Besar Badan Investasi Dunia...   
1251  Bapak Presiden Jenderal TNI (Purn) Prabowo Sub...   
1252  Mengembangkan Investasi Berbasis Teknologi den...   
1253  Masyarakat Indonesia percaya bahwa BPI DANANTA...   
1254  Masyarakat Indonesia percaya bahwa BPI DANANTA...   

                    created_at  kemunculan_serentak  
0    2025-03-13 08:30:02+00:00                    0  
1    2025-03-13 09:49:54+00:00                    0  
2    2025-03-13 07:29:41+00:00                    0  
3    2025-03-07 23:38:5

In [ ]:
# Pola Jam Aktif (Active Hours)
df['hour'] = df['created_at'].dt.hour
pola_jam_aktif = df.groupby('username')['hour'].value_counts(normalize=True).unstack(fill_value=0)
print("pola jam aktif setiap akun:")
print(pola_jam_aktif)

pola jam aktif setiap akun:
hour             0    1    2    3    4    5    6    7    8    9   ...   14  \
username                                                          ...        
03__nakula      0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
0xdermawan      0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  ...  0.0   
20detik         0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  ...  0.0   
21mntab31067    0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
2two1one        0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.2  0.0  0.0  ...  0.0   
...             ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
wilonadwasmara  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  ...  0.0   
xxxaryadi       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
yowpyoriii      0.0  0.0  0.0  0.0  0.0  0.0  0.5  0.5  0.0  0.0  ...  0.0   
yusuf_my1       0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0   
zzzeen          0.0  0.0  0.0  1.0  

/tmp/ipython-input-2799342687.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['hour'] = df['created_at'].dt.hour


In [ ]:
pola_jam_aktif

hour,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
username,,,,,,,,,,,,,,,,,,,,,
03__nakula,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0xdermawan,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20detik,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21mntab31067,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2two1one,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.8,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
wilonadwasmara,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
xxxaryadi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
yowpyoriii,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Near-Duplicate

Buat kolom baru 'is_near_duplicate' dan isi dengan False untuk semua baris di DataFrame

Untuk setiap baris i dari 0 sampai jumlah baris DataFrame: Untuk setiap baris j dari i+1 sampai jumlah baris DataFrame: Jika skor kemiripan teks baris i dan j > 0.99 DAN akun baris i berbeda dengan akun baris j DAN selisih waktu posting baris i dan j < 60 detik: Tandai is_near_duplicate pada baris i dan j sebagai True

Tampilkan kolom komentar, akun, dan is_near_duplicate dari DataFrame

In [ ]:
# Pastikan kolom 'created_at' dalam format datetime (sudah dilakukan di sel sebelumnya, tapi konversi ulang di sini untuk keamanan)
df['created_at'] = pd.to_datetime(df['created_at'])

# Buat kolom baru 'is_potential_buzzer_rule' dan isi dengan False
df['is_potential_buzzer_rule'] = False

# Tentukan ambang batas (dilonggarkan)
similarity_threshold = 0.90 # Menurunkan ambang batas kemiripan
time_threshold_seconds = 60 # Rentang waktu tetap 60 detik

for i in range(len(df)):
  for j in range(i+1, len(df)):
    # Pastikan kedua teks tidak kosong
    if pd.notna(df.loc[i, 'full_text']) and pd.notna(df.loc[j, 'full_text']):
        # Cek kemiripan teks (menggunakan cosine_sim_matrix yang sudah dihitung)
        is_similar_text = cosine_sim_matrix[i, j] > similarity_threshold

        # Cek apakah akun berbeda
        is_diff_account = df.loc[i, "username"] != df.loc[j, 'username']

        # Cek selisih waktu posting
        # Pastikan kolom created_at sudah dalam format datetime (sudah dilakukan sebelumnya)
        time_difference = abs(df.loc[i, 'created_at'] - df.loc[j, 'created_at'])
        is_close_in_time = time_difference.total_seconds() < time_threshold_seconds

        # Jika semua kriteria terpenuhi, tandai kedua baris sebagai potensial buzzer menggunakan .loc
        if is_similar_text and is_diff_account and is_close_in_time:
            df.loc[i, 'is_potential_buzzer_rule'] = True
            df.loc[j, 'is_potential_buzzer_rule'] = True

print("Dataframe dengan fitur 'is_potential_buzzer_rule':")
display(df[['full_text','username','created_at','is_potential_buzzer_rule']].head())

# Tampilkan distribusi nilai di kolom baru
print("\nDistribusi 'is_potential_buzzer_rule':")
display(df['is_potential_buzzer_rule'].value_counts())

Dataframe dengan fitur 'is_potential_buzzer_rule':


,full_text,username,created_at,is_potential_buzzer_rule
0,Mengacu dari Indonesia Salary Guide 2025 gaji ...,tempodotco,2025-03-13 08:30:02+00:00,False
1,Electrizen Danantara Indonesia telah resmi dib...,ImeldaTriS54261,2025-03-13 09:49:54+00:00,False
2,Wamenkeu Thomas: Danantara Tak Gadai Saham Pem...,cnbcindonesia,2025-03-13 07:29:41+00:00,False
3,Proyek DME: Mahal &amp; Merugikan! Gasifikasi ...,350Indonesia,2025-03-07 23:38:55+00:00,False
4,Masyarakat Indonesia percaya bahwa BPI Dananta...,liputan6dotcom,2025-03-12 10:02:10+00:00,False



Distribusi 'is_potential_buzzer_rule':


,count
is_potential_buzzer_rule,
False,1243
True,12


In [ ]:
distribusi_raw = df['is_near_duplicate'].value_counts()
print(distribusi_raw)

is_near_duplicate
False    1255
Name: count, dtype: int64


# Preprocesing & Ekstraksi Fitur

In [ ]:
import nltk
# Unduh resource NLTK yang diperlukan
try:
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    try:
        # Try downloading using the NLTK Downloader
        nltk.download('punkt_tab', quiet=True)
    except:
        # Fallback to the previous method if downloader fails
        nltk.download('punkt_tab')


try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords') # Mengunduh stopwords

try:
    nltk.data.find('taggers/averaged_perceptron_tagger')
except LookupError:
    nltk.download('averaged_perceptron_tagger')


def preprocess_teks(teks):
    """
    Fungsi untuk melakukan serangkaian preprocessing pada teks.
    """
    # 1. Lowercasing
    teks_lowercase = teks.lower()

    # 2 & 3. Menghapus tanda baca, angka, dan karakter khusus
    teks_bersih = re.sub(r'[%s]' % re.escape(string.punctuation), '', teks_lowercase)
    teks_bersih = re.sub(r'\d+', '', teks_bersih)

    # 4. Tokenisasi
    # Use word_tokenize which should now find the downloaded punkt_tab
    tokens = word_tokenize(teks_bersih)

    # 5. Menghapus stop words
    stop_words = set(stopwords.words('indonesian'))
    filtered_tokens = [kata for kata in tokens if kata not in stop_words and len(kata) > 1]

    # 6. Stemming (opsional, uncomment jika ingin digunakan)
    # stemmer = StemmerFactory().create_stemmer() # Inisialisasi stemmer jika diperlukan
    # stemmed_tokens = [stemmer.stem(kata) for kata in filtered_tokens]
    # return stemmed_tokens

    return filtered_tokens

# Daftar teks mentah yang akan diproses
daftar_teks = [
    "Saya #Sangat Senang Belajar Machine Learning! 🚀",
    "Ini adalah contoh kalimat kedua, berisi @hashtag dan angka 123.",
    "Beli buku bagus di Toko Sebelah, diskon 50%!"
]

# Perulangan (looping) untuk memproses setiap teks dalam daftar
hasil_preprocessing = []
for teks_mentah in daftar_teks:
    teks_terproses = preprocess_teks(teks_mentah)
    hasil_preprocessing.append(teks_terproses)

    print(f"Teks mentah: '{teks_mentah}'")
    print(f"Hasil preprocessing: {teks_terproses}\n")

Teks mentah: 'Saya #Sangat Senang Belajar Machine Learning! 🚀'
Hasil preprocessing: ['senang', 'belajar', 'machine', 'learning']

Teks mentah: 'Ini adalah contoh kalimat kedua, berisi @hashtag dan angka 123.'
Hasil preprocessing: ['contoh', 'kalimat', 'berisi', 'hashtag', 'angka']

Teks mentah: 'Beli buku bagus di Toko Sebelah, diskon 50%!'
Hasil preprocessing: ['beli', 'buku', 'bagus', 'toko', 'sebelah', 'diskon']



In [ ]:
# Terapkan fungsi preprocess_teks ke kolom 'full_text'
df['processed_text'] = df['full_text'].apply(preprocess_teks)

# Tampilkan beberapa baris untuk melihat hasilnya
print("DataFrame dengan kolom 'processed_text':")
display(df[['full_text', 'processed_text']].head())

DataFrame dengan kolom 'processed_text':


/tmp/ipython-input-857842092.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['processed_text'] = df['full_text'].apply(preprocess_teks)


,full_text,processed_text
0,Mengacu dari Indonesia Salary Guide 2025 gaji ...,"[mengacu, indonesia, salary, guide, gaji, sele..."
1,Electrizen Danantara Indonesia telah resmi dib...,"[electrizen, danantara, indonesia, resmi, dibe..."
2,Wamenkeu Thomas: Danantara Tak Gadai Saham Pem...,"[wamenkeu, thomas, danantara, gadai, saham, pe..."
3,Proyek DME: Mahal &amp; Merugikan! Gasifikasi ...,"[proyek, dme, mahal, amp, merugikan, gasifikas..."
4,Masyarakat Indonesia percaya bahwa BPI Dananta...,"[masyarakat, indonesia, percaya, bpi, danantar..."


# Pelabelan

In [ ]:
# Filter DataFrame untuk menampilkan baris dengan is_potential_buzzer_rule == True
potential_buzzers_by_rule = df[df['is_potential_buzzer_rule'] == True]

print("Tweet Potensial Buzzer berdasarkan Aturan (Kemiripan Teks Tinggi, Akun Berbeda, Waktu Dekat):")

if potential_buzzers_by_rule.empty:
    print("\nTidak ada tweet yang memenuhi kriteria dengan ambang batas saat ini.")
    print("Coba longgarkan ambang batas kemiripan teks atau rentang waktu jika Anda ingin mencari lebih banyak kasus.")
else:
    display(potential_buzzers_by_rule[['full_text', 'username', 'created_at', 'is_potential_buzzer_rule']])

Tweet Potensial Buzzer berdasarkan Aturan (Kemiripan Teks Tinggi, Akun Berbeda, Waktu Dekat):

Tidak ada tweet yang memenuhi kriteria dengan ambang batas saat ini.
Coba longgarkan ambang batas kemiripan teks atau rentang waktu jika Anda ingin mencari lebih banyak kasus.
